# Trabajo Preparacion, visualizacion de Datos y Machine learning con Python<a class="tocSkip">
## Ciencia de datos en Produccion <a class="tocSkip">

**Estudiante:** Sebastián Cardona y Jose Miguel Millán

**ID:** 1094910122 y 1088334182

**Email:** sacardonar@uqvirtual.edu.co y josem.millanl@uqvirtual.edu.co

 
Docente: [Jose R. Zapata](https://joserzapata.github.io)
- https://joserzapata.github.io
- https://twitter.com/joserzapata
- https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/       


## Objetivo del Trabajo <a class="tocSkip">
En el anterior experimento se entrenaron dos modelos de regresión considerando unos casos atípicos dentro del análisis, los modelos resultantes tuvieron un bajo desempeño, por lo tanto, en este experimento se eliminarán los datos atípicos encontrados.

# Definir el Problema a Resolver

El dataset "house data", inicialmente se realizará una exploración de datos, para poder saber la calidad del dataset, iniciando con una limpieza la cual consta de eliminar duplicados, identificación de datos atípicos, nullos o mal escritos para poder tratarlos y mitigarlos, ya sea con la eliminación o aplicación de métodos estadísticos, con la finalidad de tener un datset listo y poder aplicar una regresión lineal y poder predecir los precios de venta de una casa.

## Describir los datos de entrada y salida
- Cantidad de Variables
- Tipo de Variables
- Significado de cada Variable

# Importar Librerias

In [ ]:
%load_ext autoreload
%autoreload 2


from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from IPython.display import display, HTML
from pandas_profiling.profile_report import ProfileReport
from sklearn.linear_model import SGDRegressor, Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate, RepeatedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import SVR
import phik
from tqdm import tqdm

from src.data.preprocessing import preprocessing
from src.features.build_features import build_features
from src.jutils.data import DataUtils
from src.jutils.visual import Plot
from src.data.procesamiento_datos import Preprocesamiento, LimpiezaCalidad, ProcesamientoDatos
import joblib

In [ ]:
# Funciones utilizadas para realizar un preprocesamiento general.

def validar_duplicados(_df):
    _filas = _df.shape[0]
    _cant_duplicados = _df.duplicated().sum()
    print(f'De {_filas} registros hay {_cant_duplicados} filas duplicadas, representando el {_cant_duplicados/_filas:.2%}')

def eliminar_duplicados(_df):
    # Eliminando duplicados
    _df = _df.drop_duplicates(keep='first')
    _filas = _df.shape[0]
    print(f'Después de la eliminación de duplicados, el conjunto de datos queda con {_filas} filas.')
    return _df

def validar_index_duplicados(_df):
    # Validando duplicados de index
    _son_duplicados = _df['index'].duplicated()
    _cant_duplicados = _son_duplicados.sum()
    _filas = _df.shape[0]
    print(f'De {_filas} registros, hay {_cant_duplicados} registros con index duplicado, que representan el {_cant_duplicados/_filas:.2%}.')
    return _son_duplicados

def convertir_col_date_a_date(_df):
    _df['date'] = pd.to_datetime(_df['date'], errors='coerce')
    return _df

def reemplazar_valores_extremos(_df, _columnas_numericas):
    _df[_columnas_numericas] = _df[_columnas_numericas].where(lambda x: x > -1e+10, other=np.nan).where(
        lambda x: x < 1e+10, other=np.nan)
    return _df

def reemplazar_nulos_por_la_media(_df, _columnas_numericas):
    # Se reemplazan los valores nulos por la media Nota: No se considera que haya data leakage pues los valores
    # reemplazados son entre registros con el mismo index y como al final se va a dejar un dataset con index únicos,
    # no hay riesgo que estén tanto en el set de entrenamiento como en el de test
    for columna_numerica in _columnas_numericas:
        _df[columna_numerica] = _df[columna_numerica].fillna(
            _df.groupby('index')[columna_numerica].transform('median'))
    return _df

def reemplazar_fechas_nulas(_df):
    # Reemplazando fechas nulas por la primera fecha no nula
    _df['date'] = _df['date'].fillna(
        _df.groupby(['index'], sort=False)['date'].apply(lambda x: x.ffill().bfill()))
    return _df

def reemplazar_ceros_por_nulos(_df):
    # Reemplazando ceros por valores nulos
    _df[['sqft_basement', 'yr_renovated']] = _df[['sqft_basement', 'yr_renovated']].replace(0, np.nan)
    return _df



In [ ]:
# Funciones utilizadas para procesar únicamente los datos de entrenamiento
def z_score_outliers(_df, _column):
    """
    Returns:
        zscore, outlier
    """
    # Adaptado de https://www.kaggle.com/code/shweta2407/regression-on-housing-data-accuracy-87
    #creating lists to store zscore and outliers 
    zscore = []
    isoutlier =[]
    # for zscore generally taken thresholds are 2.5, 3 or 3.5 hence i took 3
    threshold = 3
    # calculating the mean of the passed column
    mean = np.mean(_df[_column])
    # calculating the standard deviation of the passed column
    std = np.std(_df[_column])
    for i in _df[_column]:
        z = (i-mean)/std
        zscore.append(z)
        #if the zscore is greater than threshold = 3 that means it is an outlier
        isoutlier.append(np.abs(z) > threshold)
    return zscore, isoutlier

In [ ]:
# Funciones para realizar el procesamiento de los datos antes de ingresar al modelo
def mediana_recortada_imputacion(_df, _column, _isoutlier):
    mediana_recortada = _df[_column][~_isoutlier].median()
    _df.loc[_isoutlier, _column] = mediana_recortada
    return _df

def calculo_variables_adicionales(_df):
    _df['tiene_sotano'] = (~_df['sqft_basement'].isna()).astype(int)
    _df['fue_renovada'] = (~_df['yr_renovated'].isna()).astype(int)
    _df['yr_date'] = _df['date'].dt.year
    _df['antiguedad_venta'] = _df['yr_date'] - _df['yr_built']
    return _df

def procesamiento_datos_faltantes(_df, _columnas):
    _df = _df.dropna(subset=_columnas)
    return _df

def clasificar_columnas(_df, _clasificacion_columnas):
    _df[_clasificacion_columnas['categorica_ordinal']] = _df[_clasificacion_columnas['categorica_ordinal']].astype(int)
    _df[_clasificacion_columnas['numerica_continua']] = _df[_clasificacion_columnas['numerica_continua']].astype(float)
    _df[_clasificacion_columnas['numerica_discreta']] = np.floor(_df[_clasificacion_columnas['numerica_discreta']])
    return _df
    
def imputacion_de_datos(_df, _columnas):
    _df[_columnas] = _df[_columnas].fillna(0)
    return _df

In [ ]:
def profiler_to_file(_profiler, archivo):
    print('Ejecutando profiler')
    _profiler.to_file(du.data_folder_path.parent.joinpath('reports/' + archivo))
    return True

def calcular_descriptivas(_df):
    descriptivas = _df.describe()
    descriptivas.loc['rango'] = descriptivas.loc['max'] - descriptivas.loc['min']
    descriptivas.loc['IQR'] = descriptivas.loc['75%'] - descriptivas.loc['25%']
    descriptivas.loc['coef de var'] = descriptivas.loc['std']/descriptivas.loc['mean']
    descriptivas.loc['skewness'] = du.data.skew(numeric_only=True)
    descriptivas.loc['kurtosis'] = du.data.kurtosis(numeric_only=True)
    return descriptivas

plot = Plot()

# Cargar Datasets

In [ ]:
du = DataUtils(
    Path(r'..\data').resolve().absolute(),
    "kc_house_dataDS.parquet",
    'price',
    lambda path: pd.read_parquet(path),
    lambda df, path: df.to_parquet(path)
)
du.data = du.load_data(du.interim_path.joinpath(du.input_file_name))

# Descripcion General del Dataset
- numero de filas y columnas
- tipos de datos y si estan correctos

Durante la exploración inicial se realizó la conversión de los tipos de datos, y la correcta representación de datos nulos.

In [ ]:
shape = du.data.shape
filas = shape[0]
columnas = shape[1]
print(f'El conjunto de datos se compone de {filas} filas y {columnas} columnas.')

In [ ]:
du.data.info()

Todas las columnas son del tipo correcto a excepción de date, se deberá hacer la conversión de este campo.

# Limpieza de calidad de datos general
- Filas con valores exactamente iguales (duplicados)
- Columnas duplicadas
- Columnas con valores constantes o sin informacion

In [ ]:
validar_duplicados(du.data)

In [ ]:
du.data = eliminar_duplicados(du.data)

In [ ]:
# Validar indices duplicados
son_duplicados = validar_index_duplicados(du.data)

In [ ]:
# Revisando los primeros registros duplicados
du.data[son_duplicados].sort_values(by='index').head()

Revisando los registros duplicados por index, se encuentra que muchas columnas tienen los mismos valores , lo único que cambia es que hay algunos faltantes y hay otros valores extremadamente bajos o altos, adicionalmente se observan algunos registros de la columna date que no son fechas. Primero se convertirá los valores de la columna date a date y los que no puedan ser convertidos se reemplazarán por valores nulos, luego se reemplazarán los valores extremos por valores nulos, luego se calculará la mediana por index para las columnas numéricas y se reemplazarán los valores nulos por estas medianas. Luego se eliminarán filas duplicadas y se reevaluarán los index duplicados.

Si nuestra suposición es correcta, no importa realizar una imputación por la mediana pues todos los valores de los índices son iguales, luego de hacer la imputación y eliminar nuevamente duplicados, no deberían quedar índices duplicados, en caso de que sigan habiendo índices duplicados se deben revertir las imputaciones realizadas y buscar otra estrategia para eliminar duplicados exactos.

In [ ]:
# Convirtiendo la columna date a datetime
du.data = convertir_col_date_a_date(du.data)

In [ ]:
# Reemplazando valores extremos, menores a -1e+10 o mayores a 1e+10
columnas_numericas = [columna for columna in du.data.columns if columna != 'date']
du.data = reemplazar_valores_extremos(du.data, columnas_numericas)

In [ ]:
# Se reemplazan los valores extremos por la media
# Nota: No se considera que haya data leakage pues los valores reemplazados son entre registros con el mismo index y como 
# al final se va a dejar un dataset con index únicos, no hay riesgo que estén tanto en el set de entrenamiento como en el de
# test
du.data = reemplazar_nulos_por_la_media(du.data, columnas_numericas)

In [ ]:
# Reemplazando fechas nulas por la primera fecha no nula
du.data = reemplazar_fechas_nulas(du.data)

Para las siguientes columnas, un cero representa un dato nulo, por lo tanto se reemplazarán.
- sqft_basement
- yr_renovated

In [ ]:
# Reemplazando ceros por valores nulos
du.data = reemplazar_ceros_por_nulos(du.data)

In [ ]:
validar_duplicados(du.data)

In [ ]:
du.data = eliminar_duplicados(du.data)

In [ ]:
son_duplicados = validar_index_duplicados(du.data)

Una vez validados los índices duplicados, se evidencia que la limpieza surtió efecto (Se debe implementar un control que valide esto cuando se vaya a realizar un reentrenamiento, se debe alertar cuando sigan habiendo índices duplicados e interrumpir el proceso)

In [ ]:
# Validando columnas con valores constantes
unicos=du.data.nunique()
unicos[unicos==1]

La columna wertyj tiene valores constantes, por lo tanto se eliminará.

In [ ]:
du.data = du.data.drop(columns=list(unicos[unicos==1].index))

In [ ]:
nulos = du.data.isnull().sum()
cant_unicos = du.data.apply(lambda x: len(x.unique()))
porce = nulos/filas
nulos = pd.DataFrame({'nulos':nulos, 'porc':porce, 'cant_unicos': cant_unicos})
# Se contarán las filas que contengan algún dato nulo
al_menos_un_nulo=du.data.isnull().any(axis=1).sum()
nulos.sort_values(by='porc', ascending=False)

In [ ]:
print(f'De {filas} registros, hay {al_menos_un_nulo} registros con al menos un valor nulo, representando el {al_menos_un_nulo/filas:.2%}')

Se debe tener en cuenta que para el caso de yr_renovated y sqft_basement, un valor nulo no representa necesariamente falta de información, para el caso de yr_renovated, un nulo representa que esa casa nunca se renovó. Y en el caso de sqft_basement quiere decir que la casa no tiene sótano.

In [ ]:
# Calculando variables adicionales
du.data = calculo_variables_adicionales(du.data)

In [ ]:
# Esta vez no se tendrán en cuenta las columnas yr_renovated y sqft_basement
df = du.data.drop(columns=['yr_renovated', 'sqft_basement'])
nulos = df.isnull().sum()
cant_unicos = df.apply(lambda x: len(x.unique()))
porce = nulos/filas
nulos = pd.DataFrame({'nulos':nulos, 'porc':porce, 'cant_unicos': cant_unicos})
# Se contarán las filas que contengan algún dato nulo
al_menos_un_nulo=df.isnull().any(axis=1).sum()
nulos.sort_values(by='porc', ascending=False)

In [ ]:
print(f'De {filas} registros, hay {al_menos_un_nulo} registros con al menos un valor nulo, representando el {al_menos_un_nulo/filas:.2%}')

Para el entrenamiento del primer modelo se eliminarán los datos nulos debido a su poca cantidad.

In [ ]:
columnas_a_eliminar_nulos = du.data.drop(columns=['yr_renovated', 'sqft_basement']).columns
du.data = procesamiento_datos_faltantes(du.data, columnas_a_eliminar_nulos)

In [ ]:
print(f'{du.data.shape=}')

## Dividir el dataset en Training set y Test set

In [ ]:
train_test, validation = train_test_split(du.data, test_size=0.2, random_state=1)
du.save_data(train_test, du.raw_train_test_path)
du.save_data(validation, du.raw_validation_path)
print(f'{train_test.shape=}')
print(f'{validation.shape=}')

# Descripcion  y Limpieza de los datos

In [ ]:
du.data = du.load_data(du.raw_train_test_path)
print('Tipos de variables')
du.data.info()

## Identificacion de Variables
- Variables de entrada y de salida
- Tipo de Variables (categoricas o Numericas)
- Tipo de datos (int, float, string, factor, boolean, ...)

In [ ]:
# Clasificación de columnas
clasificacion_columnas = {
    'categorica_ordinal': ['zipcode', 'grade', 'view', 'waterfront', 'condition', 'lat', 'long'],
    'fecha': ['date'],
    'id': ['index'],
    'numerica_continua': ['sqft_basement', 'sqft_above', 'sqft_living15', 'sqft_lot', 'price', 'sqft_lot15', 'sqft_living'],
    'numerica_discreta': ['bathrooms', 'bedrooms', 'yr_renovated', 'yr_built', 'jhygtf', 'yr_date', 'antiguedad_venta', 'floors']
}
columna_salida = 'price'
columnas_a_descartar = ['date', 'index']

du.data = du.data.drop(columns=columnas_a_descartar)

columnas_entrada = du.data.drop(columns=columna_salida).columns
print(f'Columna salida: {columna_salida}')
print(f'Columnas de entrada: {columnas_entrada}')

## Analisis General Univariable y Bivariable 
Analisis de cada una de las variables para lograr calidad de datos en cada columna
- **Correccion del tipo de dato (numericas, categoricas, string) de cada columna (optimizar memoria)**
- Deteccion de numero de datos faltantes
- Deteccion de duplicados

In [ ]:
du.data= clasificar_columnas(du.data, clasificacion_columnas)

In [ ]:
du.data.info()

# Eliminación de outliers

### Primero se analizará la distribución de cada variable

In [ ]:
box_plots = [plot.box(du.data, y=columna) for columna in clasificacion_columnas['numerica_continua']]
histograms = [plot.histogram(du.data, x=columna, text_auto=False) for columna in clasificacion_columnas['numerica_continua']]

plot.grid_subplot(*box_plots, cols= 3, title= 'Distribución inicial',titles=clasificacion_columnas['numerica_continua']).show()
plot.grid_subplot(*histograms, cols= 3, title= 'Distribución inicial',titles=clasificacion_columnas['numerica_continua']).show()

In [ ]:
# Eliminando registros identificados como outliers según el z_score

du.data = du.data[~pd.Series(z_score_outliers(du.data, 'price')[1], index=du.data.index)]
du.data = du.data[~pd.Series(z_score_outliers(du.data, 'sqft_lot')[1], index=du.data.index)]
du.data = du.data[~pd.Series(z_score_outliers(du.data, 'sqft_lot15')[1], index=du.data.index)]

In [ ]:
box_plots = [plot.box(du.data, y=columna) for columna in clasificacion_columnas['numerica_continua']]
histograms = [plot.histogram(du.data, x=columna, text_auto=False) for columna in clasificacion_columnas['numerica_continua']]

plot.grid_subplot(*box_plots, cols= 3, title= 'Distribución inicial',titles=clasificacion_columnas['numerica_continua']).show()
plot.grid_subplot(*histograms, cols= 3, title= 'Distribución inicial',titles=clasificacion_columnas['numerica_continua']).show()

Para las variables numericas_discreta se analizarán sus rangos

In [ ]:
pd.DataFrame({
    'min':du.data[clasificacion_columnas['numerica_discreta']].min(),
    'max':du.data[clasificacion_columnas['numerica_discreta']].max(),
    'nulos':du.data[clasificacion_columnas['numerica_discreta']].isna().sum()
})

Definicion de outliers:

Se considerarán outliers:
- Apartamentos con bathrooms o bedrooms igual a 0
- Apartamentos con bedrooms mayor a 5
- Apartamentos con bathrooms mayor a 4

In [ ]:
bathrooms_outliers = (du.data['bathrooms']==0) | (du.data['bathrooms'] > 4)
bedrooms_outliers = (du.data['bedrooms']==0) | (du.data['bedrooms'] > 5)
son_outliers = bathrooms_outliers | bedrooms_outliers
cant_outliers = son_outliers.sum()
print(f'Con estas características hay: {cant_outliers} outliers representando el {cant_outliers/len(son_outliers):.2%}')

Imputación de outliers: 
Se reemplazarán los outliers calculando la mediana recortada la cual se realiza teniendo en cuenta únicamente los inliers

In [ ]:
du.data = mediana_recortada_imputacion(du.data, 'bathrooms', bathrooms_outliers)
du.data = mediana_recortada_imputacion(du.data, 'bedrooms', bedrooms_outliers)

In [ ]:
# Revisando los datos después de la imputación
plot.grid_subplot(
    *[plot.bar(du.data, x=column, max_bins=10) for column in clasificacion_columnas['numerica_discreta']], 
    cols=3, 
    titles=clasificacion_columnas['numerica_discreta']).show()
pd.DataFrame({
    'min':du.data[clasificacion_columnas['numerica_discreta']].min(),
    'max':du.data[clasificacion_columnas['numerica_discreta']].max(),
    'nulos':du.data[clasificacion_columnas['numerica_discreta']].isna().sum()
})

Para las columnas categóricas se revisarán sus valores únicos.

In [ ]:
pd.DataFrame({
    'distinct_count':du.data[clasificacion_columnas['categorica_ordinal']].nunique(),
    'nulos':du.data[clasificacion_columnas['categorica_ordinal']].isna().sum()
})

No se evidencian datos atípicos. lat, long y zipcode tienen demasiada cardinalidad pero es normal por ser datos de ubicación.

## Eliminar columnas de datos Innecesarios

Se realizará un perfilado de los datos para identificar problemas de calidad no identificados anteriormente.

In [ ]:
profiler = ProfileReport(du.data, explorative=True)

profiler_to_file(profiler, '2_0_0 Perfilado inicial.html')

In [ ]:
# Se utilizará el índice de correlación phik pues este permite calcular la correlación entre variables numéricas y categóricas al tiempo.
# Este indice funciona mejor cuando no hay valores nulos, por lo tanto se reemplazarán los valores nulos por 0
cor_mat = du.data.fillna(0).phik_matrix()

In [ ]:
# Primero se analizará la correlación entre las variables de entrada y se descartarán aquellas con una correlación superior a 0.9

(cor_mat.loc[columnas_entrada, columnas_entrada] > 0.9).sum().sort_values(ascending=False)

In [ ]:
# Se analizarán con mas detalle aquellas que tienen una alta correlación con más de 1 columna (Consigo misma)
columnas_alta_correlacion = ['yr_renovated', 'fue_renovada', 'jhygtf', 'yr_built', 'tiene_sotano', 'sqft_living', 'antiguedad_venta', 'sqft_above', 'sqft_basement']
px.imshow(
    cor_mat.loc[columnas_alta_correlacion, columnas_alta_correlacion].round(2), 
    color_continuous_scale= 'blues', 
    text_auto=True).show()

La primera columna a eliminar es **jhygtf** se observa que es la misma variable que **yr_renovated**. Aunque la columna **fue_renovada** está calculada con base en **yr_renovated** se conservarán ambas para posteriormente elegir con cual de las dos se puede obtener un mejor modelo.

Quedan altas correlaciones entre las siguientes columnas:
- yr_build y antiguedad_venta
- tiene_sotano y sqft_basement
- sqft_living y sqft_above
- sqft_living y sqft_basement

Para las restantes, se conservarán todas para después mediante el cálculo de la importancia de variables escoger cual tiene un mejor poder predictivo.

Entonces solo se eliminará la columna **jhygtf**

In [ ]:
du.data = du.data.drop(columns=['jhygtf'])

In [ ]:
# Correlación de todas las variables de entrada con respecto a la salida
cor_mat.loc['price', du.data.columns].sort_values(ascending=False)

## Selección de variables
Se calculará la importancia de las variables de entrada con respecto al precio adicionando una columna dummy que contrendrá valores aleatorios y se eliminarán aquellas variables cuya importancia sea inferior a la columna aleatoria (Al ser esta aleatoria, sabemos desde el principio que esta no puede ser una buena predictora).

Para realizar este calculo se utilizará un regresor lineal, se debe escalar primero las variables de entrada para poder comparar sus coeficientes.

In [ ]:
pipeline_features = make_pipeline(StandardScaler(), LinearRegression())
X = du.data.fillna(0).drop(columns='price')
X['random'] = np.random.normal(size=(X.shape[0], 1))
y = du.data['price']

cv_model = cross_validate(
   pipeline_features, X, y, cv=RepeatedKFold(n_splits=5, n_repeats=5),
   return_estimator=True, n_jobs=2
)

coefs = pd.DataFrame(
   [model[1].coef_ for model in cv_model['estimator']],
   columns=X.columns
)
print()
print(f'mean_test_score: {np.mean(cv_model["test_score"])}')
print(f'std_test_score : {np.std(cv_model["test_score"])}')

In [ ]:
coefs

In [ ]:
px.box(coefs, orientation='h', title='Importancia de los coeficientes y sus variaciones')

In [ ]:
# Debido a que no se normalizó el precio, se dividirán los coeficientes por la media del precio para tenerlos en una escala mas
# manejable

coefs_resumen = pd.DataFrame({
    'variacion': (coefs.std()/du.data['price'].mean()),
    'media': coefs.mean()/du.data['price'].mean(),
    'media_absoluta': coefs.abs().mean()/du.data['price'].mean()
})
coef_variacion = coefs_resumen['variacion'].sort_values(ascending=False)
coef_variacion

Para aquellas variables con demasiada variación no es confiable tomar el promedio como su importancia, por lo tanto no serán eliminadas.

Se analizarán aquellas con una variación inferior a 0.7

In [ ]:
coefs_confiables = coef_variacion[coef_variacion <= 0.7]
coefs_confiables

In [ ]:
px.box(coefs[coefs_confiables.index], orientation='h', title='Importancia de los coeficientes y sus variaciones').show()
px.box(coefs.abs()[coefs_confiables.index], orientation='h', title='Importancia absoluta de los coeficientes y sus variaciones').show()

De esta forma se pueden eliminar aquellas que sean consistentemente peor que la columna random.

Al analizar el gráfico, no se pueden identificar variables con un peor desempeño que random.

Se usará f_regression de scikit learn para calcular la importancia de las variables.

In [ ]:
from sklearn.feature_selection import f_regression

f_statistic, p_values =  f_regression(X, y)
pd.DataFrame({'f_statistic': f_statistic, 'p_value': p_values.round(3)}, index=X.columns).sort_values(by='f_statistic', ascending=False)

Con esta técnica, se puede determinar el umbral desde el que se van a descartar las variables, se observa que ninguna variable tiene un poder predictivo peor que la variable random.

Pero se puede ver que la importancia de las variables **lat**, **long** y **yr_date** no son confiables al tener un p-value mayor a 0.05

Adicionalmente, podemos determinar que de las variables de entrada altamente correlacionadas, se pueden eliminar las siguientes
- antiguedad_venta > yr_built
- sqft_basement > tiene_sotano
- sqft_living > sqft_above
- sqft_living > sqft_basement

Para **yr_renovated** y **fue_renovada** la diferencia es muy poca, por lo tanto se escogerá **fue_renovada** por la alta cantidad de nulos de **yr_renovated**


Por lo tanto se descartarán las siguientes columnas:

**lat**, **long**, **yr_date**, **yr_built**, **tiene_sotano**, **sqft_above**, **sqft_basement**, **yr_built** y **yr_renovated**


In [ ]:
du.data = du.data.drop(columns=['lat', 'long', 'yr_date', 'yr_built', 'tiene_sotano', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated'])

## Procesamiento de Datos Faltantes

Ya que se eliminaron las columnas **yr_renovated** y **sqft_basement**, se puede omitir este paso pues ya no quedan mas nulos.

In [ ]:
du.data.isnull().sum().sort_values(ascending=False)

## Analisis Univariable

Estadistico Descriptico y Analisis

### Variables Numericas

| Tendencia Central |   Medida de Dispersión    | Visualizacion |
|:-----------------:|:-------------------------:|:-------------:|
|       Media       |           Rango           |  Histogramas  |
|      Mediana      |         Cuartiles         |   Boxplots    |
|       Moda        | Rango inter cuartil (IQR) |               |
|      Minimo       |         Varianza          |               |
|      Maximo       |   Desviacion Estandard    |               |
|         .         |         Skewness          |               |
|         .         |         Kurtosis          |               |

In [ ]:
calcular_descriptivas(du.data)

In [ ]:
# Actualizando la clasificación de columnas para dejar solo las que están en el dataframe
clasificacion_columnas['numerica_continua'] = list(set(clasificacion_columnas['numerica_continua']).intersection(du.data.columns))
clasificacion_columnas['categorica_ordinal'] = list(set(clasificacion_columnas['categorica_ordinal']).intersection(du.data.columns))
clasificacion_columnas['numerica_discreta'] = list(set(clasificacion_columnas['numerica_discreta']).intersection(du.data.columns))


columnas_a_graficar = clasificacion_columnas['numerica_continua']

In [ ]:
plots1 = [plot.box(du.data, y=variable_numerica) for variable_numerica in columnas_a_graficar] 
plot.grid_subplot(*plots1, cols=3, title='Diagramas de cajas y bigotes', titles=columnas_a_graficar).show()

plots2 = [plot.histogram(du.data, x=variable_numerica) for variable_numerica in columnas_a_graficar]
plot.grid_subplot(*plots2, cols=3, title='Histogramas', titles=columnas_a_graficar).show()

Se observan datos asimétricos para todas las columnas numéricas.

### Variables Categoricas
- Numero de elementos por categoria
- Porcentaje de elementos por categoria
- Graficos de barras

In [ ]:
resumen = []

for variable_categorica in clasificacion_columnas['categorica_ordinal']:
    col = du.data[variable_categorica]
    elms_cat = col.groupby(by=col).agg('count')
    total = elms_cat.sum()
    porc = elms_cat / total
    porc.name = 'porc'
    df = pd.DataFrame([elms_cat, porc]).transpose()
    resumen.append(df)

for tabla in resumen:
    display(HTML(tabla.to_html()))
    variable = tabla.columns[0]
    fig = px.bar(tabla[variable], orientation='h', title=str(variable))
    fig.show()


## Analisis Bivariable
Estadistico Descriptico y Analisis

### Numericas vs Numericas
- Scatter Plot
- Heatmap
- Correlacion

In [ ]:
columnas_a_graficar = list(filter(lambda x: x!='price', clasificacion_columnas['numerica_continua']))

plots = [plot.scatter(du.data, x=columna, y='price') for columna in columnas_a_graficar]

plot.grid_subplot(*plots, cols = 2, titles=columnas_a_graficar).show()

In [ ]:
clasificacion_columnas

In [ ]:
columnas_a_graficar = clasificacion_columnas['categorica_ordinal'] + clasificacion_columnas['numerica_discreta']
nbins = [0 if len(du.data[x].unique()) < 20 else 20 for x in columnas_a_graficar]

plots = [
    plot.box(du.data, x=column, y='price', nbins=nbin)
    for column, nbin in zip(columnas_a_graficar, nbins)
]
plot.grid_subplot(*plots, 
                  cols=2, 
                  titles=columnas_a_graficar, 
                  title='Box plot entre entradas categóricas y precio',
                  height=1500
                 ).show()

De las variables categóricas, las que parecen tener un mayor impacto en el precio son grade, view y y waterfront.
La variable condicion parece tener un aumento en el precio cuando es mayor a 3.

In [ ]:
from pandas import DataFrame

corr_matrix: DataFrame = du.data.phik_matrix()

In [ ]:
corr_matrix['price'].sort_values(ascending=False)

Las variables relacionadas con el tamaño del apartamento y la calificación tienen mayor correlación con el precio del apartamento.

In [ ]:
# Ordenando matriz de correlación con respecto a precio
corr_matrix = corr_matrix.sort_values(by='price', ascending=False)
corr_matrix = corr_matrix.reindex(columns=corr_matrix.index)
sns.heatmap(corr_matrix, cmap='PuOr')

In [ ]:
columnas = ['sqft_living', 'sqft_above', 'sqft_basement', 'antiguedad_venta', 'yr_built', 'yr_date', 'yr_renovated', 
            'fue_renovada', 'grade', 'sqft_living15', 'sqft_lot', 'tiene_sotano', 'condition', 'floors', 'bathrooms', 
            'view', 'bedrooms', 'waterfront', 'sqft_lot15', 'zipcode']

In [ ]:
profiler2 = ProfileReport(du.data, explorative=True)
profiler_to_file(profiler2, '2_0_1 Perfilado de datos.html')

In [ ]:
calcular_descriptivas(du.data)

## Analisis Univariable y Bivariable Final

In [ ]:
profiler3 = ProfileReport(du.data, explorative=True)

In [ ]:
profiler_to_file(profiler3, '2_0_2 Transformacion_final.html')
profiler3

In [ ]:
du.data.info()

### Variables de entrada
- zipcode
- grade
- view
- bathrooms
- bedrooms
- sqft_living15
- waterfront
- floors
- sqft_lot
- condition
- sqft_lot15
- sqft_living
- fue_renovada
- antiguedad_venta


### Variables de salida
- price

# MODELAMIENTO DE LOS DATOS (MACHINE LEARNING)

In [ ]:
variables_entrada = ['zipcode', 'grade', 'view', 'bathrooms', 'bedrooms', 'sqft_living15', 'waterfront', 'floors', 'sqft_lot', 'condition', 'sqft_lot15', 'sqft_living', 'fue_renovada', 'antiguedad_venta']
variable_salida  = 'price'
du.data = du.data[variables_entrada + [variable_salida]]

In [ ]:
y_name = 'price'
x_names = [columna for columna in du.data.columns if not columna == 'price']

# Validacion y Evaluacion Cruzada (k-fold Cross Validation)

Se hace seleccion de los mejores modelos usando el Training Set y k-fold Cross Validation

In [ ]:
alpha = 1
l1_ratio=0.5
normalize=False
max_iter=100000
warm_start=True
modelos_a_probar = {
    'Linear_Regression': {'modelo': make_pipeline(StandardScaler(), LinearRegression())},
    'Linear_Regression degree 2': {'modelo': make_pipeline(
        PolynomialFeatures(degree=2),
        LinearRegression()
    )},
    'Linear_Regression degree 3': {'modelo': make_pipeline(
        PolynomialFeatures(degree=3),
        LinearRegression()
    )},
    'Linear_Regression degree 2 with normalization': {'modelo': make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=2),
        LinearRegression()
    )},
    'Lasso': {'modelo': Lasso()},
    'Ridge': {'modelo': make_pipeline(
        StandardScaler(),
        Ridge()
    )},
    'Ridge degree 2': {'modelo': make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=2),
        Ridge()
    )},
    'ElasticNet': {'modelo': make_pipeline(
        StandardScaler(),
        ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=max_iter, warm_start=warm_start)
    )},
    'SGD': {'modelo': make_pipeline(StandardScaler(), SGDRegressor())},
    'SVR': {'modelo': make_pipeline(StandardScaler(), SVR())}
}

In [ ]:
for nombre_modelo, dic_modelo in tqdm(modelos_a_probar.items(), desc='Realizando cross validation...'):
    inicial = datetime.now()
    modelo = dic_modelo['modelo']
    dic_modelo['scores'] = cross_val_score(modelo, du.data[x_names], du.data[y_name], cv=5, scoring='r2')
    tiempo_entrenamiento = (datetime.now() - inicial).total_seconds()
    dic_modelo['tiempo_entrenamiento'] = tiempo_entrenamiento
    dic_modelo['media'] = np.mean(dic_modelo['scores'])
    dic_modelo['std'] = np.std(dic_modelo['scores'])

In [ ]:
tabla_comparativa = pd.DataFrame(modelos_a_probar).transpose()

In [ ]:
print('Comparativa R2')
tabla_comparativa.drop(columns=['scores','modelo']).sort_values(by='media', ascending=False)

Se escogerá el Ridge degree 2 pues aunque el regresor lineal de grado 2 con normalización obtuvo una media ligeramente mayor,
el modelo Ridge de grado 2 lo superó bastante en tiempo de entrenamiento.
Como segundo modelo para la optimización de hiper parámetros se utilizará Linear_Regression degree 2 with normalization

In [ ]:
mejor_modelo1 = modelos_a_probar['Ridge degree 2']['modelo']
mejor_modelo2 = modelos_a_probar['Linear_Regression degree 2 with normalization']['modelo']
mejor_modelo1.fit(du.data[x_names], du.data[y_name])

In [ ]:
y_predict = mejor_modelo1.predict(du.data[x_names])

In [ ]:
y_real = du.data['price']

In [ ]:
vector = np.linspace(y_real.min()*0.8, y_real.max()*1.2)
fig1 = plot.scatter(pd.DataFrame({'y_real': y_real, 'y_pred': y_predict}), x='y_real', y='y_pred')
fig2 = px.line(pd.DataFrame({'y_real': vector, 'y_pred': vector}), x='y_real', y='y_pred')
plot.combine_plots(fig2, fig1).show()

# Optimizacion de Hiper parametros (Hyper Parameter optimization)

Se seleccionan solo los mejores modelos para realizar el ajuste de hiperparametros, ya que tiene una carga computacional alta.

Al final se obtienen los parametros del mejor modelo

In [ ]:
# Obteniendo el nombre de los parámetros del primer modelo
mejor_modelo1.get_params()

In [ ]:
# Obteniendo el nombre de los parámetros del segundo modelo
mejor_modelo2.get_params()

In [ ]:
model1_param_grid = [
    {
        'polynomialfeatures__interaction_only': [True, False],
        'ridge__alpha': np.linspace(0.01, 6, 60)
    }
]

model2_param_grid = [
    {
        'polynomialfeatures__interaction_only': [True, False],
        'polynomialfeatures__include_bias': [True, False],
        'linearregression__positive': [True, False]
    }
]

In [ ]:
from sklearn.model_selection import GridSearchCV

gs1 = GridSearchCV(mejor_modelo1, model1_param_grid, scoring='r2')
gs2 = GridSearchCV(mejor_modelo2, model2_param_grid, scoring='r2')

In [ ]:
# Realizando un grid search para el primer pipeline
print('Grid search primer modelo')
gs1.fit(du.data[x_names], du.data[y_name])

print('Grid search segundo modelo')
# Realizando un grid search para el segundo pipeline
gs2.fit(du.data[x_names], du.data[y_name])
print('Listo')

In [ ]:
print('Los mejores parámetros para el primer pipeline son:')
print(gs1.best_params_)
print()
print('Los mejores parámetros para el segundo pipeline son:')
print(gs2.best_params_)

In [ ]:
from sklearn.compose import make_column_transformer

modelo_optimizado = make_pipeline(
        make_column_transformer(
            ('passthrough', [
                'zipcode', 'grade', 'view', 'bathrooms', 'bedrooms', 'sqft_living15', 'waterfront', 'floors',
                'sqft_lot', 'condition', 'sqft_lot15', 'sqft_living', 'fue_renovada', 'antiguedad_venta'
            ])
        ),
        StandardScaler(),
        PolynomialFeatures(degree=2, interaction_only=False),
        Ridge(alpha=6.0)
    )

# Evaluacion final del modelo con el Test set

Después de haber obtenido el flujo para la transformación de los datos, se empaquetó en un conjunto de pipelines de transformación.

In [ ]:
set_validacion = du.load_data(du.raw_validation_path)
set_entrenamiento = du.load_data(du.raw_train_test_path)

In [ ]:
from src.data.procesamiento_datos import Preprocesamiento

pval = Preprocesamiento(['price'], ['price'])
set_validacion_transformado = set_validacion.pipe(preprocessing).pipe(pval.transform).pipe(build_features)[[variable_salida] + variables_entrada]
set_entrenamiento_transformado = set_entrenamiento.pipe(preprocessing).pipe(pval.transform).pipe(build_features)[[variable_salida] + variables_entrada]

In [ ]:
modelo_optimizado.fit(set_entrenamiento_transformado[variables_entrada], set_entrenamiento_transformado[variable_salida])

y_real_train, y_predict_train = set_entrenamiento_transformado[variable_salida], modelo_optimizado.predict(set_entrenamiento_transformado[variables_entrada])
y_real_validation, y_predict_validation = set_validacion_transformado[variable_salida], modelo_optimizado.predict(set_validacion_transformado[variables_entrada])

In [ ]:
r2_entrenamiento = r2_score(y_real_train, y_predict_train)
r2_validacion = r2_score(y_real_validation, y_predict_validation)

print(f'{r2_entrenamiento=:.2f}')
print(f'{r2_validacion=:.2f}')

Se observa una disminución en el puntaje R2 de 4 puntos porcentuales, esto muestra que el modelo no tiene overfitting.

# Implementacion del Modelo (Deploying)
Con el análisis básico y el ajuste hecho, comienza el trabajo real (ingeniería).

El último paso para poner en produccion el modelo de prediccion sera:
1. Entrenarlo en todo el conjunto de datos nuevamente, para hacer un uso completo de todos los datos disponibles.
2. Usar los mejores parámetros encontrados mediante la validación cruzada, por supuesto. Esto es muy similar a lo que hicimos al principio, pero esta vez teniendo una idea de su comportamiento y estabilidad. La evaluación se realizó con honestidad, en divisiones distintas de entrenamiento / prueba.

El predictor final se puede serializar y grabar en el disco, de modo que la próxima vez que lo usemos, podemos omitir todo el entrenamiento y usar el modelo capacitado directamente:

In [ ]:
#import pickle # Esta es una libreria de serializacion nativa de python, puede tener problemas de seguridad

df_completo = pd.read_csv(du.raw_path.joinpath('kc_house_dataDS.csv'), index_col=0, decimal='.')
# df_transformado = df_completo.pipe(li.fit_transform).pipe(pre_pro.fit_transform).pipe(pval.fit_transform).pipe(build_features)
# df_transformado = df_transformado[[variable_salida] + variables_entrada]
_columnas_numericas = [columna for columna in df_completo.columns if columna != 'date']
pre_pro = Preprocesamiento(['price', 'sqft_lot', 'sqft_lot15'], [])
li = LimpiezaCalidad(_columnas_numericas)
pda = ProcesamientoDatos()

pipeline_tranformacion_prediccion = make_pipeline(
    li, pre_pro, pda
)

pipeline_tranformacion_entrenamiento=make_pipeline(
    li, pre_pro, pval, pda
)

df_transformado = pipeline_tranformacion_entrenamiento.fit_transform(df_completo)
pipeline_tranformacion_prediccion.fit(df_completo)
modelo_optimizado.fit(df_transformado[variables_entrada], df_transformado[variable_salida])

# Guardando
# garbar el modelo en un archivo
joblib.dump(pda, du.model_path.with_stem('pda'))

du.model = modelo_optimizado

Realizando la validación del modelo con los datos completos.

In [ ]:
from sklearn.pipeline import Pipeline
df = pd.read_csv(du.raw_path.joinpath('kc_house_dataDS.csv'), index_col=0, decimal='.')

li = LimpiezaCalidad(_columnas_numericas)
pre_pro = Preprocesamiento(['price', 'sqft_lot', 'sqft_lot15'], [])
pda: ProcesamientoDatos = joblib.load(du.model_path.with_stem('pda'))

pipeline_tranformacion_prediccion = make_pipeline(
    li, pre_pro, pda
)

pipeline_tranformacion_validacion = make_pipeline(
    li, pre_pro, pval, pda
)
model: Pipeline = joblib.load(du.model_path)

df_transformado = pipeline_tranformacion_validacion.transform(df)
y_predict = model.predict(df_transformado)
y_real = df_transformado['price']

In [ ]:
is_outlier = y_real.isna()

print(f'{r2_score(y_real[~is_outlier], y_predict[~is_outlier]):.2f}')

In [ ]:
plots = [
    plot.scatter(pd.DataFrame({'y_real_train': y_real_train, 'y_predict_train':y_predict_train}), x='y_real_train', y='y_predict_train'),
    plot.scatter(pd.DataFrame({'y_real_train': y_real, 'y_predict_train':y_predict}), x='y_real_train', y='y_predict_train')
]
plots2 = [
    plot.histogram(pd.DataFrame({'y_real_train': y_real_train, 'y_predict_train':y_predict_train}), x='y_real_train'),
    plot.histogram(pd.DataFrame({'y_real': y_real, 'y_predict_train':y_predict}), x='y_real')
]

plot.grid_subplot(*plots, 
                  cols=2, 
                  titles=['División test-train', 'Dataset completo'],
                  title='Scater plot entre datos de entrenamiento y dataset completo',
                  height=500
                 ).show()

plot.grid_subplot(*plots2, 
                  cols=2, 
                  titles=['División test-train', 'Dataset completo'],
                  title='Distribución entre datos de entrenamiento y dataset completo',
                  height=500
                 ).show()

Se observa una disminución en el R2 al entrenar con los datos completos, al revisar la distribución de los datos, se observa que en la eliminación de datos con el z-score, quedaron por fuera precios mayores de 2 millones, esto causa una disminución en el cálculo del R2. Esto será solucionado en la implementación del modelo en código.

# Comunicacion de Resultados (Data Story Telling)

### Gráfico de dependencia parcial.
Los gráficos de dependencia parcial son una forma de ver el impacto que tiene una variable en la respuesta. Se realizará el gráfico para las variables más importantes identificadas anteriormente: sqft_living, grade, sqft_above, sqft_living, bathrooms, view.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import partial_dependence
# from sklearn.inspection import plot_partial_dependence
from sklearn.inspection import PartialDependenceDisplay
from time import time

features = ['sqft_living', 'sqft_lot', 'sqft_lot15', 'grade', 'view', 'bathrooms', 'bedrooms', 'sqft_living15', 
            'waterfront', 'floors', 'condition', 'fue_renovada', 'antiguedad_venta']

x = PartialDependenceDisplay.from_estimator(model, df_transformado.drop(columns='price'),features= features)

print('Computing partial dependence plots...')


In [ ]:
fig, ax = plt.subplots(figsize=(12, 15))
x.plot(ax=ax)
fig

Se puede observar que las siguientes variables no varían mucho el precio de venta de los hogares:
- sqft_lot
- floors
- fue_renovada
- bedrooms
- bathrooms

Las siguientes tienen un efecto moderado en la salida:
- sqft_living
- sqft_living15
- condition
- view

Las siguientes variables tienen un fuerte efecto en el precio
- waterfront 
- grade
- antiguedad_venta

La siguiente variable tiene un leve impacto negativo en el precio
- sqft_lot15

# Conclusiones

- Las variables que mayor impacto tienen en el precio de los hogares son variables relacionadas con el área habitable y entorno, como por ejemplo, el diseño, el tamaño, la vista, utilizacion de los interiores, vista a fuentes hidricas.
- Se encontró que el mejor modelo es un modelo de regresión lineal con regularización (Ridge) con características polinomiales de segundo orden obteniendo un R2 de 65%, esta disminución corresponde a una eliminación exceciva de outliers, lo cual será revisado posteriormente para la mejora del modelo.
- Una correcta limpieza de outliers fue fundamental para mejorar los resultados del modelo.

**Recomendaciones**
Si actualmente se cuenta con una propiedad en Kansas y se tiene dinero disponible para invertir antes de realizar la venta, es recomendable antes de aumentar el tamaño del lote, invertir en la mejora del diseño de interior, que ayude a mejorar los espacios y aumentar el área habitable, pues se encontró que aunque la renovación aumenta el valor de las casas, si esta renovación no viene acompañada de un aumento en el valor estético de la misma, no se logran obtener los mejores beneficios.

# Ayudas Y Referencias

- https://medium.com/@joserzapata/paso-a-paso-en-un-proyecto-machine-learning-bcdd0939d387
- [Proyecto de Principio a Final sobre readmision de pacientes con Diabetes](https://github.com/JoseRZapata/Readmission-ML-Project)

- [a-complete-machine-learning-walk-through-in-python-part-one](https://towardsdatascience.com/a-complete-machine-learning-walk-through-in-python-part-one-c62152f39420)


- [a-starter-pack-to-exploratory-data-analysis-with-python-pandas-seaborn-and-scikit-learn](https://towardsdatascience.com/a-starter-pack-to-exploratory-data-analysis-with-python-pandas-seaborn-and-scikit-learn-a77889485baf#249d)

- [a-data-science-for-good-machine-learning-project-walk-through-in-python-part-one](https://towardsdatascience.com/a-data-science-for-good-machine-learning-project-walk-through-in-python-part-one-1977dd701dbc)

- [Ejemplos de Kaggle](https://www.kaggle.com/kernels?sortBy=hotness&group=everyone&pageSize=20&language=Python&kernelType=Notebook)

- [END to END ML from data colletion to deployment](https://medium.com/datadriveninvestor/end-to-end-machine-learning-from-data-collection-to-deployment-ce74f51ca203)

Docente: [Jose R. Zapata](https://joserzapata.github.io)
- https://joserzapata.github.io
- https://twitter.com/joserzapata
- https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/   